In [1]:
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import datafusion
from datafusion import SessionContext, SessionConfig, col, functions as F
import duckdb

print(duckdb.__version__)
print(datafusion.__version__)

1.3.1
47.0.0


In [2]:
batches = 100000

names = list("abcdefghijklmnopqrstuvwxyz")
names = [n + m for n in names for m in names]

names_array = pa.concat_arrays([pa.array(names)] * batches)
values_array = pa.concat_arrays([pa.array(np.random.randint(1, 100, len(names))) for _ in range(batches)])

pa_table = pa.Table.from_arrays([names_array, values_array], names=["name", "value"])

In [3]:
pa_table.shape

(67600000, 2)

In [4]:
%%timeit 
res = duckdb.sql("select name, sum(value) as value FROM pa_table group by name;").to_arrow_table()

134 ms ± 6.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
ctx = SessionContext()
_ = ctx.from_arrow(pa_table, "pa_table")

In [6]:
%%timeit
res = ctx.sql("select name, sum(value) as value FROM pa_table group by name;").to_arrow_table()

988 ms ± 135 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
